## Naive Bayes

In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

### Raw data

In [17]:
df = pd.read_csv('Data/Dropout_Academic_Success.csv', delimiter=',')
feature_names = df.columns.tolist()
target_name = 'Target'
feature_names.remove(target_name)

X = df[feature_names]
Y = df[target_name]

# Split the original dataset into training and testing sets (70:30 split)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Split the testing set into validation and testing sets (2:1 split)
X_val, X_test, Y_val, Y_test = train_test_split(X_test, Y_test, test_size=0.33, random_state=42)

In [18]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()

In [19]:
nb.fit(X_train, Y_train)

GaussianNB()

In [20]:
nb_predictions = nb.predict(X_test)
#print("Naive Bayes predictions: ", nb_predictions)

In [21]:
print("Naive Bayes score: ", nb.score(X_test, Y_test))

Naive Bayes score:  0.7107061503416856


In [22]:
from sklearn.metrics import accuracy_score

ac = accuracy_score(Y_test, nb_predictions)
print("Accuracy score: ", ac)

Accuracy score:  0.7107061503416856


In [23]:
from sklearn.metrics import f1_score

f1_sc = f1_score(Y_test, nb_predictions, average=None)
print("F1 score: ", f1_sc.round(2))

F1 score:  [0.74 0.42 0.79]


In [14]:
from sklearn.metrics import classification_report

print(classification_report(Y_test, nb_predictions))

              precision    recall  f1-score   support

     Dropout       0.77      0.72      0.74       149
    Enrolled       0.47      0.38      0.42        84
    Graduate       0.75      0.83      0.79       206

    accuracy                           0.71       439
   macro avg       0.66      0.65      0.65       439
weighted avg       0.70      0.71      0.70       439

